## Part-2 Creating the model

In [2]:
import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import graphviz



import gc
import random
import datetime
from tqdm import tqdm_notebook as tqdm


import scipy as sp
from scipy import stats
from scipy.stats import ks_2samp
from scipy.stats.mstats import gmean


from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from IPython.display import display


import lightgbm as lgb

from tqdm import tqdm_notebook as tqdm


In [3]:
# Set a few plotting defaults
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['font.size'] = 12

In [4]:
X = pd.read_pickle('I:/ML Datasets/IEEE Fradulent/train_reduced.pkl')
X_test = pd.read_pickle('I:/ML Datasets/IEEE Fradulent/test_reduced.pkl')

## SEED FUNCTION
The SEED function that defines the inital input seed

In [5]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [6]:
SEED = 42 ## This is the defining SEED
seed_everything(SEED)
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
TARGET = 'isFraud'
NFOLDS = 5

Getting the data

In [7]:
sample_submission = pd.read_csv('I:/ML Datasets/IEEE Fradulent/sample_submission.csv', index_col='TransactionID')

y = X[TARGET]
X = X.drop(TARGET, axis=1)
        
print(f'X.shape : {X.shape}, X_test.shape : {X_test.shape}')

X.shape : (590540, 434), X_test.shape : (506691, 434)


## Splitting the data

In [8]:
def train_val_split_by_time(X, y, test_size=0.2):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, shuffle=False)
    
    print(f'train.shape: {X_train.shape}, val.shape: {X_val.shape}')
    
    return X_train, y_train, X_val, y_val

In [9]:
X_train, y_train, X_val, y_val = train_val_split_by_time(X, y) ## Splitting the train data in training and validation sets

train.shape: (472432, 434), val.shape: (118108, 434)


## Plotting Feature Importance

In [10]:
def plot_feature_importance(X, model):
    feature_importance_df = pd.DataFrame()
    feature_importance_df['feature'] = X.columns.tolist()
    feature_importance_df['gain_importance'] = model.feature_importance('gain')
    feature_importance_df['split_importance'] = model.feature_importance('split')

    plt.figure(figsize=(15,10))
    plt.subplot(121)
    plot_1 = sns.barplot(x='gain_importance', y='feature',
                         data=feature_importance_df.sort_values(by='gain_importance', ascending=False)[:50])
    plot_1.set_title('LightGBM Feature Gain Importance')

    plt.subplot(122)
    plot_2 = sns.barplot(x='split_importance', y='feature',
                         data=feature_importance_df.sort_values(by='split_importance', ascending=False)[:50])
    plot_2.set_title('LightGBM Feature Split Importance')
    plt.tight_layout()
    plt.show()
    
    return feature_importance_df

In [11]:
def plot_feature_importance(fi_df):
    plt.figure(figsize=(15,10))
    plt.subplot(121)
    plot_1 = sns.barplot(x='gain_importance', y='feature',
                         data=fi_df.sort_values(by='gain_importance', ascending=False)[:50])
    plot_1.set_title('LightGBM Feature Gain Importance')

    plt.subplot(122)
    plot_2 = sns.barplot(x='split_importance', y='feature',
                         data=fi_df.sort_values(by='split_importance', ascending=False)[:50])
    plot_2.set_title('LightGBM Feature Split Importance')
    plt.tight_layout()
    plt.show()    

## LGB Parameters Setting up

In [12]:
lgb_params = {
    'objective': 'binary', ## Target vlaue is binary
    #'metric': 'auc',
    'metric': 'None',  # Default Metric: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier
    'learning_rate': 0.01, ## Learning rate
    'num_leaves': 2**8, ## number of leaves
    'max_bin': 255, ## This is the binning value per feature. It decides how many max bins to be created
    'max_depth': -1, ## This means no leaves
    'bagging_freq': 5, ## I am guessing bag at every 5th iteration
    'bagging_fraction': 0.7,  ## 70% of parameters randomly in each iteration for building trees.
    'bagging_seed': SEED, # Setting the seed
    'feature_fraction': 0.7, # Amount of features to take
    'feature_fraction_seed': SEED,
    'first_metric_only': True, 
    #Create a callback that activates early stopping.Activates early stopping. The model will train until 
    #the validation score stops improving. Validation score needs to improve at least every early_stopping_rounds
    #round(s) to continue training.Requires at least one validation data and one metric. 
    #If there’s more than one, will check all of them. But the training data is ignored anyway.
    #To check only the first metric set first_metric_only to True.
    'verbose': 100,
    'n_jobs': -1,
    'seed': SEED,
} 

## Numba 

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN. You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest.

jit is one of those decorators

In [13]:
from numba import jit

## Fast AUC

To understand the logic behind scikit's implementation of AUC-ROC curve the link is below. Note: We have to remember that 
that we are predicting the probabilites of getting a positive value and that is thhe basis of the plotting the curve.
Therefore all the probabilities are for the prediction that whether the class belongs to 1 or not. The threshold is 
randomly produced by the algorith I guess. pos means which of these are we assinging true positive.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve

https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013

The link to the fast_auc source code is given above. The explanation is not clear will be adding that when I get one

the fast_auc function is defnitely faster than the sci-kit implementation but however it hits a roadblock when say there
are two values 0.5,0.5 for the same classification of 0,1. I am not fully sure behind the logic behind the code. It seems
as if a higher weight is given for vlaues that are correctly classified at a higher threshold and then all the positive 
values are added. I dont understand the logic behind the  division 

In [14]:
@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
        print(auc)
    auc /= (nfalse * (n - nfalse))
    return auc

def eval_auc(preds, dtrain):
    labels = dtrain.get_label()
    return 'auc', fast_auc(labels, preds), True

## Covariate Shift

The whole idea behind doing covariate shift is to understand whether one variable considerablly changes between the training and the testing data. There is a shift in the distribution of the data between the training and the testing. This can be indentified by mixing the testing and the training data. Create a seperate feature that, in our case we created isTest = 0 for all the training variables and isTest = 1 for all the testing dataset. This is more of part of an EDA to understand. We use ROCAUC score and assess the only feature that we are concerned about. If the score is in the range of 0.5 it means the varialbe doesnt do a good job in distinguishing between the training and testing dataset and therefore doesnt exhibhit covariate shift.
More on this can be found at 

https://www.analyticsvidhya.com/blog/2017/07/covariate-shift-the-hidden-problem-of-real-world-data-science/

The code was taken from this thread

https://www.kaggle.com/nroman/eda-for-cis-fraud-detection



In [15]:
def covariate_shift(df_train, df_test, feature, seed=SEED):
    df_f_train = pd.DataFrame(data={feature: df_train[feature], 'isTest': 0})
    df_f_test = pd.DataFrame(data={feature: df_test[feature], 'isTest': 1})

    # Creating a single dataframe
    df = pd.concat([df_f_train, df_f_test], ignore_index=True)
    
    # Encoding if feature is categorical
    if str(df[feature].dtype) in ['object', 'category']:
        df[feature] = LabelEncoder().fit_transform(df[feature].astype(str))
    
    # Splitting it to a training and testing set
    X_train, X_test, y_train, y_test = train_test_split(df[feature], df['isTest'], test_size=0.33,
                                                        random_state=seed, stratify=df['isTest'])

    params = {
        'objective': 'binary',
        'metric': 'auc',
        'n_estimators': 500,
        'random_state': seed
    }
    
    clf = lgb.LGBMClassifier(**params)
    clf.fit(X_train.values.reshape(-1, 1), y_train)
    roc_auc =  roc_auc_score(y_test, clf.predict_proba(X_test.values.reshape(-1, 1))[:, 1])

    del df, X_train, y_train, X_test, y_test
    gc.collect();
    
    #print('feature:', feature, 'covariate shift:', roc_auc)
    
    return roc_auc

## Adversial Validation

The idea behind adversial validation is that to check each column in the train and test data together and see which columns exhibit covariate shift. The threshold is placed at 0.7 and any auc score above that means those columns have to be discarded.

Also tqdm is nothing but a progress bar that displays the amount of progress made in a for loop for each iteration. It pretty much keeps us from waiting and knowing how much of progress has been made. Below is the link for that

https://tqdm.github.io/

In [16]:
def adversarial_validation(df_train, df_test, threshold=0.7):
    list_auc_value = []

    for i in tqdm(df_train.columns.tolist()):
        auc = covariate_shift(df_test , df_train, i)
        list_auc_value.append(auc)
        if auc > threshold:
            print('feature:', i, 'covariate shift:', auc)

    cov = pd.Series(list_auc_value, index = df_train.columns.tolist()).sort_values() 
    list_discarded = list(cov[cov > threshold].index)
    
    print('features to drop:', list_discarded)
    
    return cov, list_discarded

## Frequency Encoding

https://www.kaggle.com/cdeotte/time-split-validation-malware-0-68

This helps improving the score on columns that are time dependant. I guess it has something to do with assiging weights to certain variables. I couldnt find a reasonable explanation that I could understand. I would have to dig in more.

The timeblock_frequency_encoding function does the encoding on each time period for the combination on the train and test dataset together.

In [17]:
def freq_encode_full(df1, df2, col, normalize=True):
    df = pd.concat([df1[col],df2[col]])
    vc = df.value_counts(dropna=False, normalize=normalize).to_dict()
    nm = col + '_FE_FULL'
    df1[nm] = df1[col].map(vc)
    df1[nm] = df1[nm].astype('float32')
    df2[nm] = df2[col].map(vc)
    df2[nm] = df2[nm].astype('float32')
    return nm

In [18]:
def timeblock_frequency_encoding(train_df, test_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            print('timeblock frequency encoding:', new_col)
            train_df[new_col] = train_df[col].astype(str)+'_'+train_df[period].astype(str)
            test_df[new_col]  = test_df[col].astype(str)+'_'+test_df[period].astype(str)

            temp_df = pd.concat([train_df[[new_col]], test_df[[new_col]]])
            fq_encode = temp_df[new_col].value_counts(normalize=True).to_dict()

            train_df[new_col] = train_df[new_col].map(fq_encode)
            test_df[new_col]  = test_df[new_col].map(fq_encode)
            
            train_df[new_col] = train_df[new_col]/train_df[period+'_FE_FULL']
            test_df[new_col]  = test_df[new_col]/test_df[period+'_FE_FULL']

    return train_df, test_df

## UID Aggregation

uids would be the columns.

aggregations would be mean or std

The main idea is to create a say take one column form uid and main column and take an aggregate such as mean. Both the dataframes (train and test) are joined togehter and it is called temp_df. A new column that merges the main columns and uid are created. The temp_df is grouped by the columns in uid and main column and then aggregated by either the mean or std and the 
aggrgeated column is called the inital new column that we created. This value is then mapped to the training and the testing 
dataset

In [19]:
def uid_aggregation(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                print('uid_aggregation:', new_col_name)
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
    return train_df, test_df

## Normalization

In [20]:
def values_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            print('values_normalization:', new_col)
            dt_df[col] = dt_df[col].astype(float)  

            temp_min = dt_df.groupby([period])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period].values
            temp_max = temp_max['max'].to_dict()

            temp_mean = dt_df.groupby([period])[col].agg(['mean']).reset_index()
            temp_mean.index = temp_mean[period].values
            temp_mean = temp_mean['mean'].to_dict()

            temp_std = dt_df.groupby([period])[col].agg(['std']).reset_index()
            temp_std.index = temp_std[period].values
            temp_std = temp_std['std'].to_dict()

            dt_df['temp_min'] = dt_df[period].map(temp_min)
            dt_df['temp_max'] = dt_df[period].map(temp_max)
            dt_df['temp_mean'] = dt_df[period].map(temp_mean)
            dt_df['temp_std'] = dt_df[period].map(temp_std)

            dt_df[new_col+'_min_max'] = (dt_df[col]-dt_df['temp_min'])/(dt_df['temp_max']-dt_df['temp_min'])
            dt_df[new_col+'_std_score'] = (dt_df[col]-dt_df['temp_mean'])/(dt_df['temp_std'])
            del dt_df['temp_min'],dt_df['temp_max'],dt_df['temp_mean'],dt_df['temp_std']
    return dt_df

## Feature Engineering

cols_to_drop are at the end of fe1

In [21]:
cols_to_drop = [
    'D5_DT_W_std_score',
    'ProductCD_TransactionAmt_DT_W',
    'D4_DT_D_std_score',
    'D15_DT_D_std_score',
    'D3_DT_W_std_score',
    'D11_DT_W_std_score',
    'card3_card5_DT_W_week_day_dist',
    'card5_DT_W_week_day_dist',
    'D10_DT_D_std_score',
    'card3_card5_DT_D',
    'ProductCD_cents_DT_D',
    'D4_DT_W_std_score',
    'D15_DT_W_std_score',
    'uid_DT_D',
    'card3_DT_W_week_day_dist',
    'D10_DT_W_std_score',
    'D8_DT_D_std_score',
    'card3_card5_DT_W',
    'ProductCD_cents_DT_W',
    'uid_DT_W',
    'D8_DT_W_std_score'
]

In [22]:
def fe1(df_train, df_test):
    df_tr = df_train.copy()
    df_te = df_test.copy()
    
    remove_features = [
        'TransactionID','TransactionDT'
    ]
    
    for df in [df_tr, df_te]:
        # Temporary variables for aggregation
        df['DT'] = df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
        df['DT_M'] = ((df['DT'].dt.year-2017)*12 + df['DT'].dt.month).astype(np.int8)
        df['DT_W'] = ((df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear).astype(np.int8)
        df['DT_D'] = ((df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear).astype(np.int16)

        df['DT_hour'] = (df['DT'].dt.hour).astype(np.int8)
        df['DT_day_week'] = (df['DT'].dt.dayofweek).astype(np.int8)
        df['DT_day_month'] = (df['DT'].dt.day).astype(np.int8)

        # Possible solo feature
        df['is_december'] = df['DT'].dt.month
        df['is_december'] = (df['is_december']==12).astype(np.int8)
    
    remove_features += ['DT','DT_M','DT_W','DT_D','DT_hour','DT_day_week','DT_day_month']
    
    for col in ['DT_W','DT_D']:
        freq_encode_full(df_tr, df_te, col)
        
    ## We cant use it as a solo feature
    remove_features.append(col+'_FE_FULL')
    
## Note the reason we cant use the time features individually is  because individually these features dont mean anything
## what that means is that say we new for the past 7 days the normal transactions and suddenly there is a spike in activity
## it could be due to the launch of a new iphone and therefore spike in transactions or it could be fradulent activity. 
## Without knowing what the normalcy, we dont know and cant judge. Also take a look at the test data there is a lot of missing
## content and therefore these features cannot be used independantly used since they would vary in the train and test data.
## Take a look at the discussion below to understand the reasoning
    
#https://www.kaggle.com/kyakovlev/ieee-fe-with-some-eda

    for df in [df_tr, df_te]:
        df['ProductCD_card1'] = df['ProductCD'].astype(str) + '_' + df['card1'].astype(str)
        df['card1_addr1'] = df['card1'].astype(str) + '_' + df['addr1'].astype(str)
        df['TransactionAmt_dist2'] = df['TransactionAmt'].astype(str) + '_' + df['dist2'].astype(str)
        
    remove_features.append('ProductCD_card1')
    remove_features.append('card1_addr1')
    remove_features.append('TransactionAmt_dist2')
    
## The next code is for removing the outliers from the code. The logic behind the loop is that if there are any unique 
## values that have a value less than 2 they would be removed. The train and test dataset are combined together before 
## doing the value counts. After which the values in the training and the testing values are copied from each and then from 
## the valid_card set. It should be noted that columns like (Product_CD, card1_addr1 ...Trans.._dist2) are created for 
## indentification of the UID.
    
    for col in ['card1','ProductCD_card1','card1_addr1','TransactionAmt_dist2']: 
        valid_card = pd.concat([df_tr[[col]], df_te[[col]]])
        valid_card = valid_card[col].value_counts()

        invalid_cards = valid_card[valid_card<=2]
        print('Rare data', col, len(invalid_cards))

        valid_card = valid_card[valid_card>2]
        valid_card = list(valid_card.index)

        print('No intersection in Train', col, len(df_tr[~df_tr[col].isin(df_te[col])]))
        print('Intersection in Train', col, len(df_tr[df_tr[col].isin(df_te[col])]))

        df_tr[col] = np.where(df_tr[col].isin(df_te[col]), df_tr[col], np.nan)
        df_te[col]  = np.where(df_te[col].isin(df_tr[col]), df_te[col], np.nan)

        df_tr[col] = np.where(df_tr[col].isin(valid_card), df_tr[col], np.nan)
        df_te[col]  = np.where(df_te[col].isin(valid_card), df_te[col], np.nan)
        print('#'*20)
        
## Same as the above loop but not giving a threshold rather removing values that dont correspond in each other list

    for col in ['card2','card3','card4','card5','card6']: 
        print('No intersection in Train', col, len(df_tr[~df_tr[col].isin(df_te[col])]))
        print('Intersection in Train', col, len(df_tr[df_tr[col].isin(df_te[col])]))

        df_tr[col] = np.where(df_tr[col].isin(df_te[col]), df_tr[col], np.nan)
        df_te[col]  = np.where(df_te[col].isin(df_tr[col]), df_te[col], np.nan)
        print('#'*20)
    
    # Add values remove list
    new_columns = ['uid']
    remove_features += new_columns
        
    print('#'*10)
    print('Most common uIds:')
    for col in new_columns:
        print('#'*10, col)
        print(df_tr[col].value_counts()[:10])
        
## For all the columns below we are doing frequency encoding. This is not a time feature and therefore doing the frequency
## encoding of it would be not an issue
        
    i_cols = ['card1','card2','card3','card5','ProductCD_card1','card1_addr1','TransactionAmt_dist2'] + new_columns
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
        
        
## The belwo columns we are grouping by certain columns
        
########################### card3/card5 most common hour 
# card3 or card5 is a bank country?
# can we find:
# - the most popular Transaction Hour
# - the most popular Week Day
# and then find distance from it

# Prepare bank type feature
    for df in [df_tr, df_te]:
         df['card3_card5'] = df['card3'].astype(str) +'_'+ df['card5'].astype(str)
    remove_features.append('card3_card5') 
    
    encoding_mean = {
        1: ['DT_D','DT_hour','_hour_dist','DT_hour_mean'],
        2: ['DT_W','DT_day_week','_week_day_dist','DT_day_week_mean'],
    }

    encoding_best = {
        1: ['DT_D','DT_hour','_hour_dist_best','DT_hour_best'],
        2: ['DT_W','DT_day_week','_week_day_dist_best','DT_day_week_best'],
    }

## In the below code so the first part of it combines in the first case(first loop) card3 + 'DT_D' and creates a column 
## called card3_DT_D_DT_hour_mean that then groups them by this newly created column and find the mean for the hours in each
## group. This value is then subtracted from the DT_hour.
     # Some ugly code here (even worse than in other parts)
    for col in ['card3','card5','card3_card5']:
        for df in [df_tr, df_te]:
            for encode in encoding_mean:
                encode = encoding_mean[encode].copy()
                new_col = col + '_' + encode[0] + encode[2]
                df[new_col] = df[col].astype(str) +'_'+ df[encode[0]].astype(str)

                temp_dict = df.groupby([new_col])[encode[1]].agg(['mean']).reset_index().rename(
                                                                        columns={'mean': encode[3]})
                temp_dict.index = temp_dict[new_col].values
                temp_dict = temp_dict[encode[3]].to_dict()
                df[new_col] = df[encode[1]] - df[new_col].map(temp_dict)
                
## For the second part it is similar story

            for encode in encoding_best:
                encode = encoding_best[encode].copy()
                new_col = col + '_' + encode[0] + encode[2]
                df[new_col] = df[col].astype(str) +'_'+ df[encode[0]].astype(str)
                temp_dict = df.groupby([col,encode[0],encode[1]])[encode[1]].agg(['count']).reset_index().rename(
                                                                        columns={'count': encode[3]})

                temp_dict.sort_values(by=[col,encode[0],encode[3]], inplace=True)
                temp_dict = temp_dict.drop_duplicates(subset=[col,encode[0]], keep='last')
                temp_dict[new_col] = temp_dict[col].astype(str) +'_'+ temp_dict[encode[0]].astype(str)
                temp_dict.index = temp_dict[new_col].values
                temp_dict = temp_dict[encode[1]].to_dict()
                df[new_col] = df[encode[1]] - df[new_col].map(temp_dict)
                

## The whole idea behind the block below is to check for normal activity. What that means here is take for example uid from 
## i_cols and DT_D from peroids. When they pass it onto the function. What the function does here is that it at first 
## joins both the test and train data into one single dataframe. It then creates a new column called say uid_DT_D that has
## the uid and the dates the specific user has made transanctions. This value is then counted and normalized. What that means
## is that say a specific customer on given date across different months has made how many transanctions. This value is then
## divided  by the total normalized value counts of total occurences of that specific date in the total frame. Now when we
## do that if there is a larger value this means that specific user made a large number of transactions on that specific day
## compared to the normal transactions.
########################### bank_type
# Tracking nomal activity
# by doing timeblock frequency encoding
    i_cols = ['uid','card3_card5'] #['uid','uid2','uid3','uid4','uid5','bank_type']
    periods = ['DT_W','DT_D']

# We have few options to encode it here:
# - Just count transactions
#(but some timblocks have more transactions than others)
# - Devide to total transactions per timeblock (proportions)
# - Use both
# - Use only proportions
    df_tr, df_te = timeblock_frequency_encoding(df_tr, df_te, periods, i_cols)
    
    
## This function is an aggregation of for example say uid,D1 and mean. We create a column called uid_D1_mean which at first
## combines both the test and train data. This is followed by creating a temporary dataframe that groups by column uid,
## and then does the aggregation of either mean or standard deviation over columns such as D1,D2..D16. The new dataframe 
## would be converted to a dictionary where the key would be the uid and the values would be uid_D1_mean.This would be then
## mapped to the train and test data

########################### D Columns
    # From columns description we know that
    # D1-D15: timedelta, such as days between previous transaction, etc.
    # 1. I can't imagine normal negative timedelta values (Let's clip Values)
    # 2. Normalize (Min-Max, Standard score) All D columns, except D1,D2,D9
    # 3. Do some aggregations based on uIDs
    # 4. Freaquency encoding
    # 5. D1,D2 are clipped by max train_df values (let's scale it)
    i_cols = ['D'+str(i) for i in range(1,16)]
    uids = ['uid','card3_card5']
    aggregations = ['mean','std']

    ####### uIDs aggregations
    df_tr, df_te = uid_aggregation(df_tr, df_te, i_cols, uids, aggregations)
    
    # Lets transform D8 and D9 column
    # As we almost sure it has connection with hours
    df['D9_not_na'] = np.where(df['D9'].isna(),0,1)
    df['D8_not_same_day'] = np.where(df['D8']>=1,1,0) ## Assigning all the values greater than 1 in D8 to 1 else 0
    df['D8_D9_decimal_dist'] = df['D8'].fillna(0)-df['D8'].fillna(0).astype(int)
    df['D8_D9_decimal_dist'] = ((df['D8_D9_decimal_dist']-df['D9'])**2)**0.5
    df['D8'] = df['D8'].fillna(-1).astype(int)
    
    ####### Values Normalization
    i_cols.remove('D1')
    i_cols.remove('D2')
    i_cols.remove('D9')
    periods = ['DT_D','DT_W']
    
## Here what happens is that when say a column called DT_D for period and D3 for i_cols. It groups by the period DT_D and 
## then finds the minimum value in each group for a given date. This is then saved into a dataframe temp_min with the index
## value being the dates and the values holding the minimum in each group. This is then done for max, mean  and std. These
## values are mapped for the training dataset and then min_max and std_score are calcualted. All the other columns are 
## removed except for min_max and std_score.
    
    #kaggle.com/kyakovlev/ieee-columns-scaling
    for df in [df_tr, df_te]:
        df = values_normalization(df, periods, i_cols)

    for col in ['D1','D2']:
        for df in [df_tr, df_te]:
            df[col+'_scaled'] = df[col]/df_tr[col].max()

    ####### Global Self frequency encoding
    # self_encoding=True because 
    # we don't need original values anymore
    i_cols = ['D'+str(i) for i in range(1,16)]
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
    remove_features += i_cols
    
    ####### Global Self frequency encoding
    # self_encoding=True because 
    # we don't need original values anymore
    i_cols = ['D'+str(i) for i in range(1,16)]
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
    remove_features += i_cols
    
    ########################### TransactionAmt
    # Clip Values
    ## clipping valuee because at a higher value the binning is easier for the lgbm models
    df_tr['TransactionAmt'] = df_tr['TransactionAmt'].clip(0,5000)
    df_te['TransactionAmt']  = df_te['TransactionAmt'].clip(0,5000)
    
    # For our model current TransactionAmt is a noise
    # https://www.kaggle.com/kyakovlev/ieee-check-noise
    # (even if features importances are telling contrariwise)
    # There are many unique values and model doesn't generalize well
    # Lets do some aggregations
    i_cols = ['TransactionAmt']
    uids = ['card1','card2','card3','card5','uid','card3_card5']
    aggregations = ['mean','std']

    # uIDs aggregations
    df_tr, df_te = uid_aggregation(df_tr, df_te, i_cols, uids, aggregations)
    
    
    # TransactionAmt Normalization
    periods = ['DT_D','DT_W']
    for df in [df_tr, df_te]:
        df = values_normalization(df, periods, i_cols)

    #i_cols = ['id_01','id_02','id_05','id_06','id_07','id_08','id_09','id_14']
    i_cols = ['id_01','id_02','id_05','id_06','id_09','id_14']
    uids = ['card1','card2','card3','card5','uid','card3_card5']
    aggregations = ['mean','std']

    # uIDs aggregations
    df_tr, df_te = uid_aggregation(df_tr, df_te, i_cols, uids, aggregations)
    
    
    i_cols = [
        #'V202','V203','V204','V317','V318','V257','V258',
        'V258',
        'V306','V307','V308','V294'
    ]
    uids = ['uid','card3_card5']
    aggregations = ['mean','std']
    
    # uIDs aggregations
    df_tr, df_te = uid_aggregation(df_tr, df_te, i_cols, uids, aggregations)
    
    # ProductCD, TransactionAmt
    
## Creating new columns that are the aggregations of the ProductCD and TransactionAmt
    df_tr['ProductCD_TransactionAmt'] = df_tr['ProductCD'].astype(str)+'_'+df_tr['TransactionAmt'].astype(str)
    df_te['ProductCD_TransactionAmt'] = df_te['ProductCD'].astype(str)+'_'+df_te['TransactionAmt'].astype(str)
    
## Timeblock frequency encoding is done for ProductCD_TransactionAmt with periods for DT_D and DT_W . 
    i_cols = ['ProductCD_TransactionAmt']
    periods = ['DT_D','DT_W']
    df_tr, df_te = timeblock_frequency_encoding(df_tr, df_te, periods, i_cols)

## Frequency encoding is done for ProductionCD_TransactionAmt   
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
    remove_features += i_cols
    
## Another new column is created that has the cents which is the difference between Transaction amount and its decimal part

    df_tr['cents'] = np.round( df_tr['TransactionAmt'] - np.floor(df_tr['TransactionAmt']),3 )
    df_te['cents'] = np.round( df_te['TransactionAmt'] - np.floor(df_te['TransactionAmt']),3 )

    i_cols = ['cents']
    remove_features += i_cols

## ProductCD, cents together are clubbed to create a column and then frequency encoding
    # ProductCD, cents
    df_tr['ProductCD_cents'] = df_tr['ProductCD'].astype(str)+'_'+df_tr['cents'].astype(str)
    df_te['ProductCD_cents'] = df_te['ProductCD'].astype(str)+'_'+df_te['cents'].astype(str)

    i_cols = ['ProductCD_cents']
    periods = ['DT_D','DT_W']
    df_tr, df_te = timeblock_frequency_encoding(df_tr, df_te, periods, i_cols)
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
    remove_features += i_cols
    
# Small "hack" to transform distribution 
# (doesn't affect auc much, but I like it more)
# please see how distribution transformation can boost your score 
# (not our case but related)
# https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html

    df_tr['TransactionAmt'] = np.log1p(df_tr['TransactionAmt'])
    df_te['TransactionAmt'] = np.log1p(df_te['TransactionAmt'])
    
########################### C Columns

## So in the below code all the columns from C1 to C15 is clubbed together in one single column called c_cols_0_bin. This 
## new column is then added values from each column. This is done by checking each column and seeing if the value is 0 or 1
## If the value is any value other than zero than it assigns 0 for that columns otherwise 1 for that columns. Therefore the
## the output would be astrings of 0's and 1's

    i_cols = ['C'+str(i) for i in range(1,15)]
    
    for df in [df_tr, df_te]:
        df['c_cols_0_bin'] = ''
        for c in i_cols:
            df['c_cols_0_bin'] += (df[c] == 0).astype(int).astype(str)    
    freq_encode_full(df_tr, df_te, 'c_cols_0_bin')

## In the below code fruther aggrgations are done with the sets of uid's passed onto the list and the aggregations. 
    ####### Global Self frequency encoding
    # self_encoding=False because  ## This option is not included in the function
    # I want to keep original values
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)

    uids = ['card1','card2','card3','card5','uid','card3_card5']
    aggregations = ['mean','std']

    ####### uIDs aggregations
    df_tr, df_te = uid_aggregation(df_tr, df_te, i_cols, uids, aggregations)
    
## For clipping the max values we find the max value of DT_M for each column C1-C15 and then take the max value at that 
## point for C1-C15 and that would be the upper clip for that specific column
    
    ####### Clip max values
    for df in [df_tr, df_te]:
        for col in i_cols:
            max_value = df_tr[df_tr['DT_M']==df_tr['DT_M'].max()][col].max()
            df[col] = df[col].clip(None,max_value) 
    
       
    ########################### dist1, dist2 Columns
    i_cols = ['dist1']
    uids = ['card1','card2','card3','card5','uid','card3_card5']
    aggregations = ['mean','std']

    ####### uIDs aggregations
    df_tr, df_te = uid_aggregation(df_tr, df_te, i_cols, uids, aggregations)
    
## What happens here is that, atfirst a dictionary is created called nans_group and then a dataframe nans_df
## is created with a True or False sort of a dataframe, i.e: if a given row is nan or not. Then we create a loop and 
## indentify all the columns that have Nan's greater than 0. This is then saved to the nans_group with the count of Nan as 
## the key and the column name as the count. After that another loop is done where new columns are created with the columns
## in the dictionary and the sum of those value are added,mean and std is calculated for each key and value. the columns
## V1-V340 are then removed.
    
 ####### V feature - nan group agg
    nans_groups = {}
    nans_df = pd.concat([df_tr, df_te]).isna()

    i_cols = ['V'+str(i) for i in range(1,340)]
    for col in i_cols:
        cur_group = nans_df[col].sum()
        if cur_group>0:
            try:
                nans_groups[cur_group].append(col)
            except:
                nans_groups[cur_group]=[col]

    for i, (n_group, n_cols) in enumerate(nans_groups.items()):
        print('processing nan group agg for:', n_cols)
        for df in [df_tr, df_te]:
            df[f'nan_group_{i}_sum'] = df[n_cols].sum(axis=1)
            df[f'nan_group_{i}_mean'] = df[n_cols].mean(axis=1)
            df[f'nan_group_{i}_std'] = df[n_cols].std(axis=1)

    del nans_groups, nans_df
    remove_features += i_cols
    
       ########################### Device info and identity
    for df in [df_tr, df_te]:
        ########################### Device info
        df['DeviceInfo'] = df['DeviceInfo'].fillna('unknown_device').str.lower()
        df['DeviceInfo_device'] = df['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
        df['DeviceInfo_version'] = df['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

        ########################### Device info 2
        df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
        df['id_30_device'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
        df['id_30_version'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

        ########################### Browser
        df['id_31'] = df['id_31'].fillna('unknown_device').str.lower()
        df['id_31_device'] = df['id_31'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))

    i_cols = [
        'DeviceInfo','DeviceInfo_device','DeviceInfo_version',
        'id_30','id_30_device','id_30_version',
        'id_31','id_31_device',
        'id_33',
    ]
    
    ########################### Device info and identity
    for df in [df_tr, df_te]:
        ########################### Device info
        df['DeviceInfo'] = df['DeviceInfo'].fillna('unknown_device').str.lower()
        df['DeviceInfo_device'] = df['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
        df['DeviceInfo_version'] = df['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

        ########################### Device info 2
        df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
        df['id_30_device'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
        df['id_30_version'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

        ########################### Browser
        df['id_31'] = df['id_31'].fillna('unknown_device').str.lower()
        df['id_31_device'] = df['id_31'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))

    i_cols = [
        'DeviceInfo','DeviceInfo_device','DeviceInfo_version',
        'id_30','id_30_device','id_30_version',
        'id_31','id_31_device',
        'id_33',
    ]
    
    i_cols = [
        'DeviceInfo','DeviceInfo_device','DeviceInfo_version',
        'id_30','id_30_device','id_30_version',
        'id_31','id_31_device',
        'id_33',
    ]

    ####### Global Self frequency encoding
    # self_encoding=True because 
    # we don't need original values anymore
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
    remove_features += i_cols
    
    i_cols = [
        'id_01',
        'id_03',
        'id_04',
        'id_05',
        'id_06',
        'id_07',
        'id_08',
        'id_09',
        'id_10',
        'id_11',
        'id_13',
        'id_14',
        'id_17',
        'id_18',
        'id_19',
        'id_20',
        'id_21',
        'id_22',
        'id_24',
        'id_25',
        'id_26',
    ]
    for col in i_cols:
        freq_encode_full(df_tr, df_te, col)
        
        
    # Label Encoding
    for f in df_tr.columns:
        if df_tr[f].dtype=='object' or df_te[f].dtype=='object':
            df_tr[f] = df_tr[f].fillna('unseen_before_label')
            df_te[f] = df_te[f].fillna('unseen_before_label')
            lbl = LabelEncoder()
            lbl.fit(list(df_tr[f].values) + list(df_te[f].values))
            df_tr[f] = lbl.transform(list(df_tr[f].values))
            df_te[f] = lbl.transform(list(df_te[f].values))
            df_tr[f] = df_tr[f].astype('category')
            df_te[f] = df_te[f].astype('category')

    remove_features += cols_to_drop
    print('remove_features:', remove_features)
    print(f'train.shape : {df_tr.shape}, test.shape : {df_te.shape}')
    
    ########################### Final features list
    feature_columns = [col for col in list(df_tr) if col not in remove_features]
    categorical_features = [col for col in feature_columns if df_tr[col].dtype.name == 'category']
    categorical_features = [col for col in categorical_features if col not in remove_features]
    
    return df_tr[feature_columns], df_te[feature_columns], categorical_features


## Make_val_predictions

valid_sets:  (list of Datasets or None, optional (default=None)) – List of data to be evaluated on during training

Note Lightgbm uses categorical features directly without one hot encoding hence those features are being sent directly into categorical feature

1000: boosting iterations


feval = Customized evaluation function. Should accept two parameters: preds, train_data, and return (eval_name, eval_result, is_higher_better) or list of such tuples.

Evaulation technique:
eval_namestring( The name of evaluation(wihtout whitespaces): eval_auc 

Verbose_eval:With verbose_eval = 4 and at least one item in valid_sets, an evaluation metric is printed every 4 (instead of 1) boosting stages.

Best Iteration:The index of iteration that has the best performance will be saved in the best_iteration field if early stopping logic is enabled by setting early_stopping_rounds. Note that train() will return a model from the best iteration.

In [23]:
def make_val_prediction(X_train, y_train, X_val, y_val, seed=SEED, seed_range=3, lgb_params=lgb_params,
                        category_cols=None):
    print(X_train.shape, X_val.shape)

    train_data = lgb.Dataset(X_train, label=y_train) ## Saves memory therefore loading into a lgb.Dataset
    val_data = lgb.Dataset(X_val, label=y_val)
    
    auc_arr = []
    best_iteration_arr = []
    val_preds = np.zeros((X_val.shape[0], seed_range)) ## Array of the format (118108, 3)
    
    feature_importance_df = pd.DataFrame()
    feature_importance_df['feature'] = X_train.columns.tolist()
    feature_importance_df['gain_importance'] = 0
    feature_importance_df['split_importance'] = 0
    
    for i, s in enumerate(range(seed, seed + seed_range)):
        seed_everything(s)
        params = lgb_params.copy()  ## Creating a copy of the lgb_params defined above 
        params['seed'] = s # Assinging that seed to those parameters
        params['bagging_seed'] = s
        params['feature_fraction_seed'] = s

        clf = lgb.train(params, train_data, 10000, valid_sets = [train_data, val_data], categorical_feature=category_cols,
                        early_stopping_rounds=500, feval=eval_auc, verbose_eval=200)
         

        best_iteration = clf.best_iteration
        best_iteration_arr.append(best_iteration)
        val_pred = clf.predict(X_val, best_iteration)
        val_preds[:, i] = val_pred
        
        auc = fast_auc(y_val, val_pred)
        auc_arr.append(auc)
        print('seed:', s, ', auc:', auc, ', best_iteration:', best_iteration)

        feature_importance_df['gain_importance'] += clf.feature_importance('gain')/seed_range
        feature_importance_df['split_importance'] += clf.feature_importance('split')/seed_range

    auc_arr = np.array(auc_arr)
    best_iteration_arr = np.array(best_iteration_arr)
    best_iteration = int(np.mean(best_iteration_arr))

    avg_pred_auc = fast_auc(y_val, np.mean(val_preds, axis=1))
    print(f'avg pred auc: {avg_pred_auc:.5f}, avg auc: {np.mean(auc_arr):.5f}+/-{np.std(auc_arr):.5f}, avg best iteration: {best_iteration}')

    feature_importance_df = feature_importance_df.sort_values(by='split_importance', ascending=False).reset_index(drop=True)
    plot_feature_importance(feature_importance_df)
    display(feature_importance_df.head(20))
    
    return feature_importance_df, best_iteration, val_preds

In [24]:
%%time
X_train, X_val, category_cols1 = fe1(X_train, X_val)


Rare data card1 5134
No intersection in Train card1 20399
Intersection in Train card1 452033
####################
Rare data ProductCD_card1 10509
No intersection in Train ProductCD_card1 33115
Intersection in Train ProductCD_card1 439317
####################
Rare data card1_addr1 21640
No intersection in Train card1_addr1 57867
Intersection in Train card1_addr1 414565
####################
Rare data TransactionAmt_dist2 29957
No intersection in Train TransactionAmt_dist2 70254
Intersection in Train TransactionAmt_dist2 402178
####################
No intersection in Train card2 6102
Intersection in Train card2 466330
####################
No intersection in Train card3 146
Intersection in Train card3 472286
####################
No intersection in Train card4 0
Intersection in Train card4 472432
####################
No intersection in Train card5 7339
Intersection in Train card5 465093
####################
No intersection in Train card6 45
Intersection in Train card6 472387
###############

processing nan group agg for: ['V143', 'V144', 'V145', 'V150', 'V151', 'V152', 'V159', 'V160', 'V164', 'V165', 'V166']
processing nan group agg for: ['V167', 'V168', 'V172', 'V173', 'V176', 'V177', 'V178', 'V179', 'V181', 'V182', 'V183', 'V186', 'V187', 'V190', 'V191', 'V192', 'V193', 'V196', 'V199', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216']
processing nan group agg for: ['V169', 'V170', 'V171', 'V174', 'V175', 'V180', 'V184', 'V185', 'V188', 'V189', 'V194', 'V195', 'V197', 'V198', 'V200', 'V201', 'V208', 'V209', 'V210']
processing nan group agg for: ['V217', 'V218', 'V219', 'V223', 'V224', 'V225', 'V226', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V235', 'V236', 'V237', 'V240', 'V241', 'V242', 'V243', 'V244', 'V246', 'V247', 'V248', 'V249', 'V252', 'V253', 'V254', 'V257', 'V258', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278']
processing nan group

In [25]:

#fi_df, best_iteration1, val_preds = make_val_prediction(X_train, y_train, X_val, y_val, category_cols=category_cols1)

In [26]:
train_data = lgb.Dataset(X_train, label=y_train) ## Saves memory therefore loading into a lgb.Dataset
val_data = lgb.Dataset(X_val, label=y_val)
auc_arr = []
best_iteration_arr = []
val_preds = np.zeros((X_val.shape[0], 1)) ## Array of the format (118108, 3)
    
feature_importance_df = pd.DataFrame()
feature_importance_df['feature'] = X_train.columns.tolist()
feature_importance_df['gain_importance'] = 0
feature_importance_df['split_importance'] = 0


In [ ]:
%%time
s = 42
seed_range = 1
seed_everything(s)
params = lgb_params.copy()  ## Creating a copy of the lgb_params defined above 
params['seed'] = s # Assinging that seed to those parameters
params['bagging_seed'] = s
params['feature_fraction_seed'] = s


clf = lgb.train(params, train_data, 10000, valid_sets = [train_data, val_data], categorical_feature=category_cols1,
                early_stopping_rounds= 10, feval=eval_auc, verbose_eval=200)
         

best_iteration = clf.best_iteration
best_iteration_arr.append(best_iteration)
val_pred = clf.predict(X_val, best_iteration)
val_preds[:, 1] = val_pred
feature_importance_df['gain_importance'] += clf.feature_importance('gain')/seed_range
feature_importance_df['split_importance'] += clf.feature_importance('split')/seed_range

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
22361.0
25245.0
25245.0
25245.0
25245.0
25245.0
25245.0
25245.0
25245.0
25245.0
25245.0


82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
82756.0
87574.0
87574.0
87574.0
87574.0
87574.0
87574.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
92397.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0
97251.0


210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
210218.0
2

410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
410229.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
419831.0
429478.0
429478.0
429478.0
429478.0
429478.0
429478.0
429478.0
429478.0
429478.0
429478.0
4

...

1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1094762194.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0
1095166467.0

1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0
1111766298.0

1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0
1123137551.0

1153357437.0
1153357437.0
1153357437.0
1153357437.0
1153357437.0
1153357437.0
1153357437.0
1153357437.0
1153357437.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.0
1153767158.

1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0
1168948483.0

1181708892.0
1181708892.0
1181708892.0
1182121001.0
1182121001.0
1182121001.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182533112.0
1182945238.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.0
1183357364.01225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225550964.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0
1225966444.0


In [ ]:
feature_importance_df = feature_importance_df.sort_values(by='split_importance', ascending=False).reset_index(drop=True)
plot_feature_importance(feature_importance_df)
display(feature_importance_df.head(20))